# Run Aging Model

In [1]:
import pybamm # loads pybamm package
import matplotlib.pyplot as plt # package for plotting
import numpy as np # for arrays
import pandas as pd # for structure use .csv for importing and exporting
import math # log, sin, exp
from scipy.integrate import solve_ivp # integration, used in accelerated simulation
import pickle # for saving simulations
import os, sys # path stuff
#sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

import os

cwd = os.getcwd()
cwd



'/home/xinlei/Anna_deg_paper/PyBaMM-deg-model-pub/degradation_model'

In [2]:
eSOH_DIR = "./data/eSOH/"
oCV_DIR = "./data/ocv/"
cyc_DIR = "./data/cycling/"
fig_DIR = "./figures/"
res_DIR = "./results/"
resistance_DIR = "./data/resistance/"
%matplotlib widget

In [3]:
# loading parameter values without aging model parameters, just battery parameters
parameter_values = get_parameter_values()
# we are defing the model
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "irreversible",
        "stress-induced diffusion": "false",
    }
)
param=spm.param

In [4]:
eSOH_DIR = "./data/eSOH/"
oCV_DIR = "./data/ocv/"
cells = [1, 4, 7, 10, 13, 16]

for cell in cells:
    cell_no = f'{cell:02d}'
    eSOH_file = eSOH_DIR + "aging_param_cell_" + cell_no + ".csv"
    oCV_file = oCV_DIR + "ocv_data_cell_" + cell_no + ".csv"
    print("Trying to read:", eSOH_file, "and", oCV_file)
    cell_no, dfe, dfe_0, dfo_0, N, N_0 = load_data(cell, eSOH_DIR, oCV_DIR)
    Ns = np.insert(N[1:] - 1, 0, 0)
    eps_n_data, eps_p_data, c_rate_c, c_rate_d, dis_set, Temp, SOC_0 = init_exp(cell_no, dfe, spm, parameter_values)


Trying to read: ./data/eSOH/aging_param_cell_01.csv and ./data/ocv/ocv_data_cell_01.csv
Trying to read: ./data/eSOH/aging_param_cell_04.csv and ./data/ocv/ocv_data_cell_04.csv
Trying to read: ./data/eSOH/aging_param_cell_07.csv and ./data/ocv/ocv_data_cell_07.csv
Trying to read: ./data/eSOH/aging_param_cell_10.csv and ./data/ocv/ocv_data_cell_10.csv
Trying to read: ./data/eSOH/aging_param_cell_13.csv and ./data/ocv/ocv_data_cell_13.csv
Trying to read: ./data/eSOH/aging_param_cell_16.csv and ./data/ocv/ocv_data_cell_16.csv


## Need to run the below cell only once to generate and save simulations. The figures can then be produced from saved simulations

In [5]:
cells = [1,4,7,10,13,16]
for cell in cells:
    cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
    Ns = np.insert(N[1:]-1,0,0)
    eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
    print(cell_no)
    pybamm.set_logging_level("WARNING")

    experiment = pybamm.Experiment(
        [
            ("Discharge at "+c_rate_d+dis_set,
            "Rest for 10 sec",
            "Charge at "+c_rate_c+" until 4.2V", 
            "Hold at 4.2V until C/100")
        ] *N[-1],
        termination="50% capacity",
    #     cccv_handling="ode",
    )
    parameter_values = get_parameter_values()
    parameter_values.update(
        {
            "Negative electrode active material volume fraction": eps_n_data,
            "Positive electrode active material volume fraction": eps_p_data,
            "Initial temperature [K]": 273.15+Temp,
            "Ambient temperature [K]": 273.15+Temp,
            "Positive electrode LAM constant proportional term [s-1]": 4.0312e-08,
            "Negative electrode LAM constant proportional term [s-1]": 1.8157e-07,
            "Positive electrode LAM constant proportional term 2 [s-1]": -1.4406e-09,
            "Negative electrode LAM constant proportional term 2 [s-1]": -4.9170e-09,
            "Positive electrode LAM constant exponential term": 1.0776,
            "Negative electrode LAM constant exponential term": 1.0776,
            "SEI kinetic rate constant [m.s-1]":  4.608e-16,
            "EC diffusivity [m2.s-1]": 4.56607447e-19,#8.30909086e-19,
            "SEI growth activation energy [J.mol-1]": 1.874e+04,
            "Lithium plating kinetic rate constant [m.s-1]": 2.3586e-09,
            "Initial inner SEI thickness [m]": 0e-09,
            "Initial outer SEI thickness [m]": 5e-09,
            "SEI resistivity [Ohm.m]": 30000.0,
            "Negative electrode partial molar volume [m3.mol-1]": 7e-06,
            "Negative electrode LAM min stress [Pa]": 0,
            "Negative electrode LAM max stress [Pa]": 0,
            "Positive electrode LAM min stress [Pa]": 0,
            "Positive electrode LAM max stress [Pa]": 0,
        },
        check_already_exists=False,
    )
    if cell == 13 or cell == 16:
        parameter_values.update(
            {
                "Negative electrode partial molar volume [m3.mol-1]":	0.747*7e-06,
            },
            check_already_exists=False,
        )
    # Aging simulations
    all_sumvars_dict = cycle_adaptive_simulation_V2(spm, parameter_values, experiment,SOC_0, save_at_cycles=1)
    with open(res_DIR+'fast_sim_'+"cell_"+cell_no+'_sum_var.pickle', 'wb') as handle:
        pickle.dump(all_sumvars_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    # Resistance simulations
    Rs = []
    Rs_s = []
    for cyc_no in range(len(Ns)):
        Rs_t,Rs_s_t = get_Rs(cyc_no,all_sumvars_dict,parameter_values,Ns,spm)
        Rs.append(Rs_t)
        Rs_s.append(np.round(Rs_s_t,4))
    print(Rs)
    df = pd.DataFrame({'N': N,'Ah_th':dfe["Ah_th"]-dfe["Ah_th"][0], 'Rs_data': dfe["Rs_ave"],'Rs_sim':Rs,'Rs_s':Rs_s
                })
    df.to_csv(res_DIR + "DC_resistance"+"_cell_"+cell_no+".csv", index=False)
    # Voltage and Expansion simulations
    for cyc_no in [0,int((len(N)+1)/2)-1,int((len(N)+1)/2),len(N)-2,len(N)-1]:
        t_d,V_d,I_d,Q_d,E_d = load_cycling_data_ch(cell,eSOH_DIR,oCV_DIR,cyc_DIR,cyc_no)
        t,I,Q,Vt,Exp,sol,rmse_V,rmse_E,rmse_VQ,rmse_EQ = cyc_comp_ch(cyc_no,all_sumvars_dict,t_d,Q_d,V_d,E_d,parameter_values,spm,Ns,c_rate_c,c_rate_d)
        df = pd.DataFrame({'t': t,'I': I, 'Q': Q,'Vt':Vt,'Exp':Exp,
                        })
        df.to_csv(res_DIR + "volt_exp_sim_ch_cell_"+cell_no+"_cyc_"+f"{N[cyc_no]}"+".csv", index=False)


01


CasADi - 2024-07-02 00:37:37 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:515]


[0.009125610331559232, 0.00942244965511813, 0.010112896871589555, 0.010775100291889607, 0.01156645050219386, 0.012402259707245959, 0.013287546374640559, 0.014228540577963352, 0.015233081678251156, 0.016310996923911747]
04
[0.009247170613221093, 0.009684135854239016, 0.010205834752663186, 0.010822223833275812, 0.011543758986729156, 0.012384792697566226, 0.01336463507914792, 0.01450959738671559]
07
[0.009432625665290278, 0.009941448894012779, 0.010577803932409216, 0.011355359585437888, 0.01229250472369801, 0.013416110150464997]
10
[0.009304066683383915, 0.00977223674812095, 0.010285798179836644, 0.010841034660105966, 0.011439705043863633, 0.0120822041050885, 0.012769016477562675, 0.01350095948779598, 0.014065794506507015, 0.014878963648184475, 0.01574156483066859]
13
[0.009270341059005326, 0.009739935675242332, 0.01023013500550024, 0.010734850999543376, 0.011256907903333866, 0.011797343301032652, 0.012357069842866055, 0.01293714097321914, 0.01353879011456423, 0.014163450038782673, 0.0148